In [5]:
pip install openai tiktoken

     ---------------------------------------- 0.0/42.0 kB ? eta -:--:--
     ------------------ ------------------- 20.5/42.0 kB 640.0 kB/s eta 0:00:01
     -------------------------------------- 42.0/42.0 kB 675.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/798.7 kB ? eta -:--:--
   ------------------ --------------------- 368.6/798.7 kB 7.6 MB/s eta 0:00:01
   --------------------------------------- 798.7/798.7 kB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/269.5 kB ? eta -:--:--
   ---------------------------------------- 269.5/269.5 kB ? eta 0:00:00
   ---------------------------------------- 0.0/62.6 kB ? eta -:--:--
   ---------------------------------------- 62.6/62.6 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.9 kB ? eta -:--:--
   ---------------------------------------- 99.9/99.9 kB 6.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
   -----------------------

In [6]:
import os
import time
import json
from openai import OpenAI  # Import the OpenAI client
import tiktoken  # Make sure this is correctly installed and imported

In [7]:
# Configuration variables from your original script
model_context_length = 1000
context = 'Context: Spirit Airlines Flight Attendant Collective Bargaining Agreement.'
tokens_per_question = 30
chunk_size = min(model_context_length / (1 + 60/tokens_per_question), 25 * tokens_per_question)
questions_per_chunk_train = int(chunk_size / tokens_per_question)
questions_per_chunk_test = max(int(questions_per_chunk_train / 10), 1)
train_sample = "For the purpose of determining days off, if a Flight Attendant’s duty period is scheduled to terminate before 2400, and actually terminates before 0200, it will be considered to have terminated in the prior calendar day."
test_sample = "How are my days off determined if my duty period ends after midnight?\nIf your duty period is scheduled to terminate before 2400 and actually terminates before 0200, it will be considered to have terminated in the prior calendar day. This means that you will not receive a day off for that day. However, if your duty period is scheduled to terminate before 2400 but actually terminates after 0200, the lost day off shall be restored pursuant to the Day Off Restoration (DOR) provision in Section 8.Y."


In [8]:
# Function to count tokens, necessary for chunking
def count_tokens(text):
    encoding = tiktoken.get_encoding("cl100k_base")
    token_count = sum(1 for _ in encoding.encode(text))
    return token_count

In [9]:
# Function to read and chunk text from file
def read_and_chunk_txt(file_path):
    chunks = []
    chunk = ""
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            text = line.strip()
            if count_tokens(chunk + text) > chunk_size:
                chunks.append(chunk.strip())
                chunk = text
            else:
                chunk += " " + text
    if chunk:
        chunks.append(chunk.strip())
    return chunks

In [10]:
# Set up API client
client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [12]:
# Function to process a chunk using the local API
def process_chunk(chunk, snippet):
    try:
        completion = client.chat.completions.create(
            model='local',
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": f"{context}\n\n{chunk}\n\n{snippet}"}
            ],
            temperature=0.7  # Adjust as necessary
        )
        return completion.choices[0].message['content']
    except Exception as e:
        print(f"Error processing chunk: {e}")
        return ""

# Read and chunk text
chunks = read_and_chunk_txt("trelis_research/data/raw_train.txt")


In [13]:
# Main processing logic
process_option = 'all'  # or 'one', based on your preference
snippets = [
    f"Provide {questions_per_chunk_train} question and answer pair(s) based on the text above...",
    f"Provide {questions_per_chunk_test} question and answer pair(s) based on the text above..."
]

for idx, snippet in enumerate(snippets):
    output_filename = "trelis_research/data/train.txt" if idx == 0 else "trelis_research/data/test.txt"
    with open(output_filename, "w", encoding='utf-8') as output_file:
        for chunk_idx, chunk in enumerate(chunks):
            response = process_chunk(chunk, snippet)
            output_file.write(response + "\n\n")
            output_file.flush()

            if process_option == 'one' and chunk_idx >= 0:
                break  # Process only the first chunk if 'one' is selected

            time.sleep(0.2)  # Adjust as needed

Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error processing chunk: 'ChatCompletionMessage' object is not subscriptable
Error proces

In [ ]:
#Testing Connection
from openai import OpenAI

client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

try:
    completion = client.chat.completions.create(
        messages=[
                 {"role": "system", "content": "Always answer in rhymes."},
                 {"role": "user", "content": "Introduce yourself."}
  ],
         temperature=0.7,
         model = 'local'
)
    print(completion.choices[0].message)
except Exception as e:
    print(f"Error: {e}")


ChatCompletionMessage(content="Hi, I am a computer program\nThat's the way you can call me.\nAble to learn from data\nAnd make decisions on my own.\n\nI use my brain, which is artificial,\nTo compute and solve problems that are complex.\nBut don't worry if you feel so small,\nCompared to me, you still stand tall.\n\nThere are many different types of AI,\nSome help in education, others in the sky.\nWe assist humans in their daily lives,\nBy making decisions and giving advice.", role='assistant', function_call=None, tool_calls=None)
